In [141]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix, classification_report, f1_score
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.display.max_rows = None
pd.set_option('future.no_silent_downcasting', True)

In [2]:
import numpy as np
print(np.__version__)

1.26.4


In [69]:
df = pd.read_csv("../Downloads/st_export.csv").iloc[:,1:]
for i in range(len(df)):
    df.iloc[i,-3]=df.iloc[i,-3].split("GMT")[0].strip()
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [91]:
a=np.random.randint(len(df))
df.iloc[a:a+5,:]

,klineacc,spread,spreadper,x,vwap,deviation,ratio,term,sigma,e,h,asset,timestamp,gap,gaplimit
93446,100250536,-5.710986e+06,-48.566540,-0.793446,91256.523438,-0.002732,1.485665,1.187643,0.002019,-2.386726,-0.983239,BTC,2025-11-28 09:41:09,5m,60
93447,10624884,-1.438284e+05,-3.342660,-0.855477,896.125061,-0.000329,1.033427,1.525232,0.001660,-0.312593,-0.302795,BNB,2025-11-28 09:43:22,3m,100
93448,33493132,4.428978e+06,20.695633,0.707756,141.103104,-0.009660,0.793044,0.738676,0.005845,-0.968237,-0.747928,SOL,2025-11-28 09:43:22,3m,100
93449,22113486,-6.474776e+05,-15.281739,-0.826950,489.767273,-0.011306,1.152817,0.932625,0.009952,-1.221469,-0.840087,ZEC,2025-11-28 09:43:22,3m,100
93450,86119240,-2.493419e+06,-35.220108,-0.827435,3016.544922,-0.003502,1.352201,1.370381,0.004157,-1.561147,-0.915606,ETH,2025-11-28 09:43:22,3m,100


In [212]:
ndf = pd.read_csv("../Downloads/st_exportd.csv").iloc[:,1:]
for i in range(len(ndf)):
    ndf.iloc[i,-3]=ndf.iloc[i,-3].split("GMT")[0].strip()
ndf['timestamp'] = pd.to_datetime(ndf['timestamp'])
a=np.random.randint(len(ndf))
ndf.iloc[a:a+5,:]

,klineacc,spread,spreadper,x,vwap,deviation,ratio,term,sigma,e,h,asset,timestamp,gap,gaplimit
10593,26561406,6.347698e+05,24.232344,0.833414,5.179189,0.020430,0.757677,0.348794,0.017934,0.301051,0.292274,ICP,2025-11-04 21:13:46,5m,60
10594,49509080,-5.238540e+06,-120.289795,-0.752206,2.266795,0.001767,2.202898,0.259299,0.003714,0.271695,0.265201,XRP,2025-11-04 21:13:46,5m,60
10595,19245970,-9.000852e+05,-47.339378,-0.804420,2.045270,0.000895,1.473394,0.279303,0.006083,0.060534,0.060460,SUI,2025-11-04 21:13:46,5m,60
10596,57053672,3.996601e+06,42.243893,0.770961,952.341003,-0.005713,0.577561,0.181282,0.004298,-0.139192,-0.138300,BNB,2025-11-04 21:13:46,5m,60
10597,352511168,4.503882e+06,36.823338,0.856473,104043.570312,-0.001878,0.631767,0.395834,0.002757,-0.170315,-0.168687,BTC,2025-11-04 21:13:46,5m,60


In [252]:
train_df=ndf.iloc[:110254,:]
actual_x=ndf.iloc[110254:,:]
len(train_df),len(actual_x)

(110254, 11939)

In [253]:
actual_x.tail()

,klineacc,spread,spreadper,x,vwap,deviation,ratio,term,sigma,e,h,asset,timestamp,gap,gaplimit
122188,114687704,-1.139888e+06,-11.481897,-0.864609,3200.404541,0.002926,1.114819,0.552623,0.002327,0.774857,0.649745,ETH,2025-12-11 17:27:58,1m,300
122189,137269296,3.348898e+06,25.213015,0.832530,90198.750000,0.001883,0.747870,0.520180,0.001141,0.641714,0.566065,BTC,2025-12-11 17:27:58,1m,300
122190,40195824,-6.068756e+06,-45.375259,-0.723352,130.829987,0.002293,1.453753,0.307849,0.002763,0.371380,0.355198,SOL,2025-12-11 17:27:58,1m,300
122191,37996272,-7.182824e+04,-2.378636,-0.899389,414.056946,-0.001128,1.023786,1.147885,0.013148,-0.100802,-0.100462,ZEC,2025-12-11 17:27:58,1m,300
122192,19820596,-2.800598e+06,-72.215736,-0.729029,867.964783,-0.001330,1.722157,0.733399,0.003229,-0.520363,-0.477980,BNB,2025-12-11 17:27:58,1m,300


In [249]:
assets_to_train=[i for i in actual_x["asset"].unique().tolist() if i in train_df["asset"].unique().tolist()]
print(assets_to_train)

['SUI', 'XRP', 'LINK', 'ZEC', 'SOL', 'ETH', 'BNB', 'BTC', 'AVAX', 'EGLD', 'BCH', 'AAVE', 'WBTC', 'TON', 'ORCA', 'ZRO', 'COMP', 'ETC', 'AR', 'INJ', 'ORDI', 'CAKE', 'PAXG', 'TRB', 'LTC', 'GIGGLE', 'NEAR', 'RENDER', 'TAO', 'DOT', 'DASH', 'QNT', 'FIL', 'PENDLE', 'ATOM', 'UNI', 'TRUMP', 'ICP', 'ZEN', 'APT', 'NMR', 'WBETH']


In [250]:
c=[[len(train_df[train_df["asset"]==i]),i] for i in assets_to_train]

In [251]:
c = sorted(c, key=lambda x: x[0],reverse=True)
print(c)

[[10202, 'BTC'], [10182, 'ETH'], [9608, 'SOL'], [9597, 'ZEC'], [9521, 'BNB'], [9472, 'XRP'], [4852, 'SUI'], [3692, 'ICP'], [3306, 'DASH'], [3204, 'GIGGLE'], [3055, 'LTC'], [2954, 'LINK'], [2887, 'ZEN'], [2449, 'UNI'], [2403, 'NEAR'], [2345, 'TAO'], [2146, 'FIL'], [1918, 'TRUMP'], [1760, 'AVAX'], [1301, 'PAXG'], [1212, 'BCH'], [796, 'AAVE'], [661, 'AR'], [567, 'DOT'], [480, 'ETC'], [359, 'CAKE'], [359, 'RENDER'], [348, 'ORDI'], [344, 'WBTC'], [326, 'INJ'], [311, 'APT'], [288, 'TON'], [288, 'PENDLE'], [265, 'ATOM'], [250, 'TRB'], [204, 'NMR'], [194, 'WBETH'], [191, 'QNT'], [185, 'ZRO'], [125, 'ORCA'], [82, 'COMP'], [28, 'EGLD']]


In [255]:
ndf["gap"].unique()

array(['1m', '3m', '5m', '15m', '30m', '1h', '2h', '4h', '6h', '8h',
       '12h'], dtype=object)

# new

In [144]:
# train_and_evaluate_pipeline.py
import pandas as pd
import numpy as np
import os
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
from datetime import timedelta

OUT_DIR = Path("./eval_outputs")
MODELS_DIR = OUT_DIR / "asset_models"
PRED_CSV = OUT_DIR / "predictions_nndf_with_metrics.csv"
PER_ASSET_CSV = OUT_DIR / "per_asset_metrics.csv"
PER_ASSET_GAP_CSV = OUT_DIR / "per_asset_gap_metrics.csv"

EVAL_MODE = "time_window"   # "first_k" or "time_window"
FIRST_K = 5                 # used if first_k mode
MIN_TRAIN_ROWS_PER_ASSET = 20  # warning if too small, but still attempts training

RANDOM_STATE = 42
N_ESTIMATORS = 200

# ---------- utility helpers ----------
def parse_gap_to_minutes(gap_str):
    # common formats: '1m','3m','5m','15m','30m','1h','2h','4h','6h','8h','12h','1d','1w'
    if pd.isna(gap_str):
        return np.nan
    s = str(gap_str).strip().lower()
    if s.endswith('m'):
        return int(s[:-1])
    if s.endswith('M'):
        return int(s[:-1]) * 60 * 24 * 30
    if s.endswith('h'):
        return int(s[:-1]) * 60
    if s.endswith('d'):
        return int(s[:-1]) * 60 * 24
    if s.endswith('w'):
        return int(s[:-1]) * 60 * 24 * 7
    # fallback try int
    try:
        return int(s)
    except:
        return np.nan

def ensure_dirs():
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    MODELS_DIR.mkdir(parents=True, exist_ok=True)

# ---------- load & preprocess ----------
print("Loading CSVs...")
df_raw = train_df.copy()   # preserve your 'do not edit variable names' pattern if needed
ndf_raw = actual_x.copy()

# parse timestamp columns
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])
ndf_raw['timestamp'] = pd.to_datetime(ndf_raw['timestamp'])

# build nndf: rows in ndf after the last index row from df (your definition)
# (you used: nndf = ndf.iloc[df.tail(1).index.start+1:,:] — we replicate robustly)
last_df_index_in_ndf = None
# find the last (by timestamp) row in df and then select ndf rows with timestamp > that
last_df_ts = df_raw['timestamp'].max()
nndf = ndf_raw[ ndf_raw['timestamp'] > last_df_ts ].copy().reset_index(drop=True)
print(f"train_df rows: {len(df_raw):,}, full_df rows: {len(ndf_raw):,}, nndf rows: {len(nndf):,}")

# add gap_minutes and gaplimit (ensure gaplimit exists)
if 'gap' in df_raw.columns:
    df_raw['gap_min'] = df_raw['gap'].apply(parse_gap_to_minutes)
    nndf['gap_min'] = nndf['gap'].apply(parse_gap_to_minutes)
else:
    raise ValueError("No 'gap' column found in df — pipeline requires it.")

if 'gaplimit' not in df_raw.columns:
    df_raw['gaplimit'] = df_raw.get('gaplimit', np.nan)
if 'gaplimit' not in nndf.columns:
    nndf['gaplimit'] = nndf.get('gaplimit', np.nan)

# ---------- compute true_next_vwap for training ----------
# true_next_vwap is the next row's vwap within the same (asset, gap) group
print("Computing true_next_vwap in df (group by asset,gap sorted by timestamp)...")
df_sorted = df_raw.sort_values(['asset','gap','timestamp']).copy()
df_sorted['true_next_vwap'] = df_sorted.groupby(['asset','gap'])['vwap'].shift(-1)
train_df = df_sorted.dropna(subset=['true_next_vwap']).copy()  # rows that have a true next vwap
print("train_df rows (those with a valid next vwap):", len(train_df))

# ---------- feature selection ----------
# select numeric columns except target and meta columns:
meta_cols = set(['asset','timestamp','gap','gaplimit','gap_min','true_next_vwap'])
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
features = [c for c in numeric_cols if c not in meta_cols]
# ensure gap_min and gaplimit included
if 'gap_min' not in features:
    features.append('gap_min')
if 'gaplimit' not in features:
    features.append('gaplimit')
# dedupe
features = list(dict.fromkeys(features))
print("Using features:", features)

# ---------- training: per-asset model ----------
ensure_dirs()
assets_to_train = sorted(list(set(nndf['asset'].unique()).intersection(set(df_raw['asset'].unique()))))
print("Assets to forecast (intersection):", assets_to_train)

asset_models = {}
per_asset_train_metrics = []

for asset in assets_to_train:
    rows = train_df[ train_df['asset'] == asset ]
    if len(rows) < 1:
        print(f"--- Asset: {asset} -> no training rows (skip).")
        continue

    X = rows[features].fillna(0.0).values
    y = rows['true_next_vwap'].values

    # split for a quick validation
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.15, random_state=RANDOM_STATE)
    model = RandomForestRegressor(n_estimators=N_ESTIMATORS, random_state=RANDOM_STATE, n_jobs=-1)
    model.fit(X_tr, y_tr)

    y_pred_val = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred_val)
    mae = mean_absolute_error(y_val, y_pred_val)
    r2 = r2_score(y_val, y_pred_val)

    per_asset_train_metrics.append({
        'asset': asset,
        'train_rows': len(rows),
        'val_RMSE': rmse,
        'val_MAE': mae,
        'val_R2': r2
    })

    model_path = MODELS_DIR / f"rf_{asset}.joblib"
    joblib.dump({'model':model, 'features':features}, model_path)
    asset_models[asset] = {'model':model, 'features':features, 'model_path':str(model_path)}
    # print(f"--- Asset: {asset} --- Trained & saved -> {model_path}  (train rows: {len(rows)})")

per_asset_train_df = pd.DataFrame(per_asset_train_metrics)
per_asset_train_df.to_csv(OUT_DIR / "per_asset_train_metrics.csv", index=False)

# ---------- produce one prediction per (asset, gap) using the latest df row for that (asset,gap) ----------
print("\nProducing one prediction per (asset,gap) from latest df rows...")
pred_records = []

# define gaps we consider: union of gaps present in df for these assets (expected 11 per asset in ideal case)
gaps_universe = sorted(df_raw['gap'].dropna().unique().tolist(), key=lambda x: str(x))

for asset in assets_to_train:
    model_info = asset_models.get(asset)
    if model_info is None:
        print(f"Asset {asset} has no trained model. Skipping predictions for this asset.")
        continue
    model = model_info['model']
    feats = model_info['features']

    # for each gap available in the universe, try to create a prediction (one per gap)
    asset_df = df_raw[ (df_raw['asset']==asset) ].sort_values(['gap','timestamp'])
    # find last row per gap
    for gap in gaps_universe:
        sub = asset_df[ asset_df['gap'] == gap ]
        if sub.empty:
            # no last-row for this gap
            continue
        last_row = sub.sort_values('timestamp').iloc[-1]
        X_row = last_row[feats].fillna(0.0).values.reshape(1,-1)
        pred = model.predict(X_row)[0]

        pred_records.append({
            'asset': asset,
            'gap': gap,
            'gap_min': parse_gap_to_minutes(gap),
            'gaplimit': last_row.get('gaplimit', np.nan),
            'last_df_timestamp': last_row['timestamp'],
            'last_df_vwap': last_row['vwap'],
            'pred_next_vwap': float(pred),
            'model_path': model_info['model_path']
        })

pred_df = pd.DataFrame(pred_records)
print("Predictions produced:", len(pred_df))

# ---------- evaluation vs nndf ----------
print("\nEvaluating predictions against nndf using mode:", EVAL_MODE)
eval_rows = []

if len(nndf) == 0:
    print("nndf is empty. No evaluation possible. Predictions saved though.")
else:
    # make nndf indexed by asset & gap for quick selection
    nndf_sorted = nndf.sort_values('timestamp').copy()

    for _, prow in pred_df.iterrows():
        asset = prow['asset']
        gap = prow['gap']
        last_ts = prow['last_df_timestamp']
        gap_min = prow['gap_min']
        gaplimit = prow['gaplimit']
        pred_val = prow['pred_next_vwap']
        last_vwap = prow['last_df_vwap']

        # candidate true rows selection:
        cand = nndf_sorted[(nndf_sorted['asset']==asset) & (nndf_sorted['gap']==gap) & (nndf_sorted['timestamp'] > last_ts)]

        if EVAL_MODE == "first_k":
            cand_sel = cand.head(FIRST_K)
        else:  # time_window
            # window = gap_min * gaplimit (minutes) into the future
            # If gaplimit NaN, fall back to FIRST_K behavior
            if pd.isna(gap_min) or pd.isna(gaplimit):
                cand_sel = cand.head(FIRST_K)
            else:
                window_minutes = int(gap_min * gaplimit)
                window_end = last_ts + timedelta(minutes=window_minutes)
                cand_sel = cand[(cand['timestamp'] <= window_end)]

        if cand_sel.empty:
            eval_rows.append({
                'asset': asset, 'gap': gap, 'count_truths': 0,
                'pred_next_vwap': pred_val, 'rmse': np.nan, 'mae': np.nan, 'direction_acc': np.nan
            })
            continue

        truths = cand_sel['vwap'].values.astype(float)
        errors = truths - pred_val
        rmse = np.sqrt(np.mean(errors**2))
        mae = np.mean(np.abs(errors))
        # direction accuracy: compare sign(true - last_vwap) vs sign(pred - last_vwap)
        pred_dir = np.sign(pred_val - last_vwap)
        true_dirs = np.sign(truths - last_vwap)
        dir_acc = np.mean(true_dirs == pred_dir)

        eval_rows.append({
            'asset': asset,
            'gap': gap,
            'count_truths': len(truths),
            'pred_next_vwap': pred_val,
            'rmse': float(rmse),
            'mae': float(mae),
            'direction_acc': float(dir_acc),
            'last_df_timestamp': last_ts,
            'window_end_if_time_mode': (last_ts + timedelta(minutes=int(gap_min*gaplimit))).isoformat() if (EVAL_MODE=='time_window' and (not pd.isna(gap_min) and not pd.isna(gaplimit))) else None
        })

eval_df = pd.DataFrame(eval_rows)

# summarize per-asset metrics by averaging across gaps (weight by count_truths)
if not eval_df.empty:
    agg_records = []
    for asset, g in eval_df.groupby('asset'):
        valid = g.dropna(subset=['rmse','mae','direction_acc'])
        if valid.empty:
            agg_records.append({'asset':asset,'RMSE':np.nan,'MAE':np.nan,'Direction_Acc':np.nan,'Count':int(g['count_truths'].sum())})
            continue
        # weighted by counts where appropriate
        counts = valid['count_truths'].values
        rmse_w = np.average(valid['rmse'], weights=counts)
        mae_w = np.average(valid['mae'], weights=counts)
        diracc_w = np.average(valid['direction_acc'], weights=counts)
        agg_records.append({'asset':asset,'RMSE':float(rmse_w),'MAE':float(mae_w),'Direction_Acc':float(diracc_w),'Count':int(g['count_truths'].sum())})
    per_asset_metrics_df = pd.DataFrame(agg_records).sort_values('RMSE', ascending=False)
else:
    per_asset_metrics_df = pd.DataFrame(columns=['asset','RMSE','MAE','Direction_Acc','Count'])

# write outputs
ensure_dirs()
pred_full = pred_df.merge(eval_df, on=['asset','gap'], how='left', suffixes=('','_eval'))
pred_full.to_csv(PRED_CSV, index=False)
per_asset_metrics_df.to_csv(PER_ASSET_CSV, index=False)
eval_df.to_csv(PER_ASSET_GAP_CSV, index=False)

print("\nOutputs saved:")
print(" - Predictions + per-gap eval: ", PRED_CSV)
print(" - Per-asset metrics (aggregated): ", PER_ASSET_CSV)
print(" - Per-asset-gap eval table: ", PER_ASSET_GAP_CSV)
print("Pipeline complete.")

Loading CSVs...
train_df rows: 118,032, full_df rows: 118,649, nndf rows: 617
Computing true_next_vwap in df (group by asset,gap sorted by timestamp)...
train_df rows (those with a valid next vwap): 117408
Using features: ['klineacc', 'spread', 'spreadper', 'x', 'vwap', 'deviation', 'ratio', 'term', 'sigma', 'e', 'h', 'gap_min', 'gaplimit']
Assets to forecast (intersection): ['AAVE', 'APT', 'AR', 'AVAX', 'BCH', 'BNB', 'BTC', 'CAKE', 'DASH', 'DOT', 'EGLD', 'ETC', 'ETH', 'FIL', 'GIGGLE', 'ICP', 'INJ', 'LINK', 'LTC', 'NEAR', 'NMR', 'ORDI', 'PAXG', 'RENDER', 'SOL', 'SUI', 'TAO', 'TON', 'TRUMP', 'UNI', 'WBTC', 'XRP', 'ZEC', 'ZEN']

Producing one prediction per (asset,gap) from latest df rows...
Predictions produced: 369

Evaluating predictions against nndf using mode: time_window

Outputs saved:
 - Predictions + per-gap eval:  eval_outputs/predictions_nndf_with_metrics.csv
 - Per-asset metrics (aggregated):  eval_outputs/per_asset_metrics.csv
 - Per-asset-gap eval table:  eval_outputs/per_a

In [146]:
pred_full[pred_full["asset"]=="ETH"].tail(11)

,asset,gap,gap_min,gaplimit,last_df_timestamp,last_df_vwap,pred_next_vwap,model_path,count_truths,pred_next_vwap_eval,rmse,mae,direction_acc,last_df_timestamp_eval,window_end_if_time_mode
134,ETH,1d,1440,35,2025-11-01 22:51:01,4079.863281,4050.242120,eval_outputs/asset_models/rf_ETH.joblib,0,4050.242120,NaN,NaN,NaN,NaT,NaN
135,ETH,1h,60,5,2025-12-09 18:44:25,3113.358398,3101.343811,eval_outputs/asset_models/rf_ETH.joblib,1,3101.343811,18.645935,18.645935,0.000000,2025-12-09 18:44:25,2025-12-09T23:44:25
136,ETH,1m,1,300,2025-12-09 19:17:41,3112.141113,3111.537220,eval_outputs/asset_models/rf_ETH.joblib,23,3111.537220,20.077392,12.903092,0.000000,2025-12-09 19:17:41,2025-12-10T00:17:41
137,ETH,1w,10080,5,2025-11-01 22:51:21,4132.763672,4121.312195,eval_outputs/asset_models/rf_ETH.joblib,0,4121.312195,NaN,NaN,NaN,NaT,NaN
138,ETH,2h,120,60,2025-12-09 17:38:40,3099.807373,3104.574141,eval_outputs/asset_models/rf_ETH.joblib,1,3104.574141,8.858076,8.858076,0.000000,2025-12-09 17:38:40,2025-12-14T17:38:40
139,ETH,30m,30,10,2025-12-09 18:50:50,3113.953857,3111.977024,eval_outputs/asset_models/rf_ETH.joblib,1,3111.977024,9.269070,9.269070,0.000000,2025-12-09 18:50:50,2025-12-09T23:50:50
140,ETH,3m,3,100,2025-12-09 19:04:14,3112.696777,3112.821633,eval_outputs/asset_models/rf_ETH.joblib,13,3112.821633,9.785949,7.542114,0.923077,2025-12-09 19:04:14,2025-12-10T00:04:14
141,ETH,4h,240,30,2025-12-09 17:05:14,3099.679199,3107.235259,eval_outputs/asset_models/rf_ETH.joblib,0,3107.235259,NaN,NaN,NaN,NaT,NaN
142,ETH,5m,5,60,2025-12-09 19:16:08,3112.519531,3112.694114,eval_outputs/asset_models/rf_ETH.joblib,15,3112.694114,15.292509,9.834369,1.000000,2025-12-09 19:16:08,2025-12-10T00:16:08
143,ETH,6h,360,20,2025-12-09 10:50:49,3101.489502,3103.232581,eval_outputs/asset_models/rf_ETH.joblib,0,3103.232581,NaN,NaN,NaN,NaT,NaN


In [155]:
# ------------------- EVALUATION -------------------
from datetime import timedelta
import numpy as np
import pandas as pd

FIRST_K = 5  # fallback for missing gap_min/gaplimit

eval_rows = []

for _, prow in pred_df.iterrows():
    asset = prow['asset']
    gap = prow['gap']
    last_ts = prow['last_df_timestamp']
    gap_min = prow['gap_min']
    gaplimit = prow['gaplimit']
    pred_val = prow['pred_next_vwap']
    last_vwap = prow['last_df_vwap']

    # select candidate rows from existing nndf in memory
    cand = nndf[(nndf['asset'] == asset) &
                (nndf['gap'] == gap) &
                (nndf['timestamp'] > last_ts)]
    # print(asset,cand)

    # if pd.isna(gap_min) or pd.isna(gaplimit):
        # cand_sel = cand.head(FIRST_K)
    # else:
    GAP_TOTAL_MIN = {
        '1m': 300,
        '3m': 300,
        '5m': 300,
        '15m': 300,
        '30m': 300,
        '1h': 300,
        '2h': 7200,
        '4h': 7200,
        '6h': 7200,
        '8h': 7200,
        '12h': 7200,
        '1d':50400,
        '1w':50400,
        '1M': 216000
    }
    total_min = GAP_TOTAL_MIN[gap]
    window_end = last_ts + timedelta(minutes=total_min)
    # window_end = last_ts + timedelta(minutes=int(gap_min * gaplimit))
    cand_sel = cand[cand['timestamp'] <= window_end]

    if cand_sel.empty:
        eval_rows.append({
            'asset': asset,
            'gap': gap,
            'count_truths': 0,
            'pred_next_vwap': pred_val,
            'rmse': np.nan,
            'mae': np.nan,
            'direction_acc': np.nan
        })
        continue

    truths = cand_sel['vwap'].values.astype(float)
    errors = truths - pred_val
    rmse = np.sqrt(np.mean(errors**2))
    mae = np.mean(np.abs(errors))

    # direction accuracy
    pred_dir = np.sign(pred_val - last_vwap)
    true_dirs = np.sign(truths - last_vwap)
    dir_acc = np.mean(true_dirs == pred_dir)

    eval_rows.append({
        'asset': asset,
        'gap': gap,
        'count_truths': len(truths),
        'pred_next_vwap': pred_val,
        'rmse': float(rmse),
        'mae': float(mae),
        'direction_acc': float(dir_acc)
    })

eval_df = pd.DataFrame(eval_rows)

# ------------------- AGGREGATE PER-ASSET -------------------
agg_records = []
for asset, g in eval_df.groupby('asset'):
    valid = g.dropna(subset=['rmse', 'mae', 'direction_acc'])
    if valid.empty:
        agg_records.append({
            'asset': asset,
            'RMSE': np.nan,
            'MAE': np.nan,
            'Direction_Acc': np.nan,
            'Count': int(g['count_truths'].sum())
        })
        continue
    counts = valid['count_truths'].values
    rmse_w = np.average(valid['rmse'], weights=counts)
    mae_w = np.average(valid['mae'], weights=counts)
    diracc_w = np.average(valid['direction_acc'], weights=counts)
    agg_records.append({
        'asset': asset,
        'RMSE': float(rmse_w),
        'MAE': float(mae_w),
        'Direction_Acc': float(diracc_w),
        'Count': int(g['count_truths'].sum())
    })

per_asset_metrics_df = pd.DataFrame(agg_records).sort_values('RMSE', ascending=False)

# ------------------- RESULTS -------------------
print("Per-prediction evaluation:")
display(eval_df.dropna(subset=['rmse', 'mae', 'direction_acc']))
print("\nPer-asset aggregated metrics:")
display(per_asset_metrics_df)

Per-prediction evaluation:


,asset,gap,count_truths,pred_next_vwap,rmse,mae,direction_acc
4,AAVE,2h,1,189.673788,0.093927,0.093927,1.000000
14,APT,2h,1,1.788389,0.012108,0.012108,0.000000
25,AR,2h,1,4.049468,0.004290,0.004290,1.000000
36,AVAX,2h,1,13.698936,0.036085,0.036085,1.000000
47,BCH,2h,1,582.801558,0.792342,0.792342,1.000000
55,BNB,15m,2,887.574644,1.083316,1.016936,0.000000
58,BNB,1h,1,887.934339,1.819409,1.819409,0.000000
59,BNB,1m,23,886.262606,2.354893,1.102778,0.391304
61,BNB,2h,1,895.034722,1.539971,1.539971,0.000000
62,BNB,30m,1,886.565345,0.246558,0.246558,0.000000



Per-asset aggregated metrics:


,asset,RMSE,MAE,Direction_Acc,Count
6,BTC,233.770365,140.891108,0.321429,56
30,WBTC,39.842109,39.842109,1.000000,1
12,ETH,15.415644,10.447603,0.517857,56
32,ZEC,8.945182,7.442273,0.642857,56
26,TAO,2.180940,2.180940,1.000000,1
5,BNB,1.657783,0.862377,0.500000,56
4,BCH,0.792342,0.792342,1.000000,1
22,PAXG,0.767996,0.767996,1.000000,1
24,SOL,0.510154,0.263525,0.107143,56
18,LTC,0.174470,0.174470,0.000000,1
